<img src="./cat_herder.png" alt="Alt text" style="width:100px; float:left; margin-right:10px;"/>

<div style="overflow: auto;">
    <h1> Cat Herder - Simple Debugger using LLM Reflection
<p><p>
    Galen Wilkerson</h1>

    One of the most common tasks using LLMs is for programming.

    However, for several reasons, LLMs are not perfect programmers (yet). Very often, you get the code back and it has bugs, either problems with import statements, runtime errors, etc., and you find yourself re-submitting your code with the errors and hoping the LLM gets it right.

    This program lets you:

    * modify existing code or do something new
        * it asks you how many debug iterations you would like
        * it then accesses ChatGPT to write the program
        * executes the program locally
        * detects if there were any errors
    * re-submits the new code with the errors
        * if no errors, it stops iterating
</div>


In [12]:
import openai
import os

def read_api_key(file_path):
    """
    Reads the API key from a specified file.

    Parameters:
    file_path (str): The path to the file containing the API key.

    Returns:
    str: The API key.
    """
    with open(file_path, 'r') as file:
        return file.read().strip()

def extract_code(response, code_type='python'):
    """
    Extracts the code from the OpenAI API response based on the specified type.

    Parameters:
    response (dict): The response dictionary from the OpenAI API.
    code_type (str): The type of code block (default is 'python').

    Returns:
    str: The extracted code.
    """
    # Extract the message content
    content = response['choices'][0]['message']['content']

    # Find the start and end of the code block
    start_tag = f"```{code_type}\n"
    start_index = content.find(start_tag) + len(start_tag)
    end_index = content.find("\n```", start_index)

    # Extract the code
    code = content[start_index:end_index]
    
    return code

def call_chatgpt(prompt, client, code_type='python', special_instructions='Implement a script in a single code block to perform this task: '):
    """
    Calls the OpenAI API to generate code based on the provided prompt.

    Parameters:
    prompt (str): The prompt describing the desired code.
    client (openai.OpenAI): The OpenAI client.
    code_type (str): The type of code block (default is 'python').

    Returns:
    str: The generated code.
    """
    special_instructions = f"Implement a {code_type} script in a single code block to perform this task: "
    response = client.chat.completions.create(
        #         model="gpt-3.5-turbo",
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": special_instructions + prompt}
        ]
    )
    # print(response)  # Add this line to inspect the response structure
    return extract_code(response.dict(), code_type=code_type)

def debug_code_with_chatgpt(original_prompt, code, error, client, code_type='python'):
    """
    Calls the OpenAI API to debug the provided code based on the encountered error.

    Parameters:
    original_prompt (str): The original prompt describing the desired code.
    code (str): The code that produced an error.
    error (str): The error message.
    client (openai.OpenAI): The OpenAI client.
    code_type (str): The type of code block (default is 'python').

    Returns:
    str: The suggested fix for the code.
    """
    prompt = f"The original prompt was:\n\n{original_prompt}\n\nHere is the code:\n\n{code}\n\nIt produced the following error:\n\n{error}\n\nPlease help me fix it."
    return call_chatgpt(prompt, client, code_type=code_type)

def save_code_to_file(code, code_type, iteration, status):
    """
    Saves the provided code to a specified file.

    Parameters:
    code (str): The code to be saved.
    code_type (str): The type of code (e.g., python, latex, html).
    iteration (int): The iteration number of the code.
    status (str): The status of the code (e.g., iteration number or 'debugged').
    """
    # Ensure the scripts directory exists
    if not os.path.exists('./scripts'):
        os.makedirs('./scripts')

    # Determine the file extension based on the code type
    ext = 'py' if code_type == 'python' else code_type

    file_path = f"./scripts/script_{status}.{ext}"
    with open(file_path, 'w') as file:
        file.write(code)

def main():
    """
    Main function to read API key, generate code, and debug it iteratively.
    """
    api_key = read_api_key('api_key.txt')
    openai.api_key = api_key
    client = openai

    choice = input("Do you want to (1) enter a new code prompt or (2) modify existing code? Enter 1 or 2: ")
    
    if choice == '1':
        user_prompt = input("Please enter the code prompt: ")
        code_type = input("Please enter the code type (default: python, latex, html, etc.): ") or 'python'
        code = call_chatgpt(user_prompt, client, code_type)
    elif choice == '2':
        file_path = input("Please enter the path to the existing code file: ")
        with open(file_path, 'r') as file:
            code = file.read()
        user_prompt = input("Please describe the modifications you want to make: ")
        code_type = input("Please enter the code type (default: python, latex, html, etc.): ") or 'python'
        code = call_chatgpt(user_prompt, client, code_type, special_instructions='Modify the following code:\n\n' + code + '\n\n')
    else:
        print("Invalid choice.")
        return

    if code_type == 'latex':
        save_code_to_file(code, code_type, 0, 'initial')
        print(f"Code saved to ./scripts/script_initial.{code_type}")
        return

    iterations = int(input("How many debug iterations should be done? "))

    print('------------------')
    print("Initial code generated by ChatGPT:\n", code)
    print('------------------')

    save_code_to_file(code, code_type, 0, 'initial')

    last_error_message = None

    for i in range(1, iterations + 1):
        print(f"\nIteration {i}:")

        try:
            if code_type == 'python':
                exec(code)
            else:
                print("Execution not supported for this code type in the current implementation.")
                break
            print("Code executed successfully.")
            save_code_to_file(code, code_type, i, 'debugged')
            print(f'Saved to ./scripts/script_debugged.{code_type}')
            break
        except Exception as e:
            error_message = str(e)
            print("Error encountered:", error_message)

            if error_message == last_error_message:
                print("Encountered the same error. Stopping iterations.")
                break
            last_error_message = error_message
            
            # Get help from ChatGPT
            code = debug_code_with_chatgpt(user_prompt, code, error_message, client, code_type)
            print('------------------')
            print("ChatGPT suggests the following fix:\n", code)
            print('------------------')

            save_code_to_file(code, code_type, i, f'iteration_{i}')

    # Save the last version of the code
    save_code_to_file(code, code_type, iterations, f'final')
    # Determine the file extension based on the code type
    ext = 'py' if code_type == 'python' else code_type
    print(f"Final version saved to ./scripts/script_final.{ext}")

if __name__ == "__main__":
    main()


Do you want to (1) enter a new code prompt or (2) modify existing code? Enter 1 or 2: 1
Please enter the code prompt: make this into a script to run from the command line, with -h and other input parameters:  import openai import os  def read_api_key(file_path):     """     Reads the API key from a specified file.      Parameters:     file_path (str): The path to the file containing the API key.      Returns:     str: The API key.     """     with open(file_path, 'r') as file:         return file.read().strip()  def extract_code(response, code_type='python'):     """     Extracts the code from the OpenAI API response based on the specified type.      Parameters:     response (dict): The response dictionary from the OpenAI API.     code_type (str): The type of code block (default is 'python').      Returns:     str: The extracted code.     """     # Extract the message content     content = response['choices'][0]['message']['content']      # Find the start and end of the code block     

usage: ipykernel_launcher.py [-h] [--prompt PROMPT] [--type TYPE]
                             [--debug DEBUG]
                             file_path
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

/home/galen/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
